In [5]:
#Saturated Oxygen Data Correction

In [27]:
#This is to import libraries of different functions that allows the program do what it's doing
%pylab inline
from netCDF4 import Dataset
import pandas as pd
from pandas import Series , DataFrame
import csv
from IPython.core.display import Image
import urllib2
import StringIO
from thredds_crawler.crawl import Crawl
import seawater

Populating the interactive namespace from numpy and matplotlib


In [7]:
data = urllib2.urlopen("http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt").readlines()
data[1].replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '')
#So this works for a specific row but not the whole thing O.o
#Make a loop that runs through all of the rows one by one (since all together doesn't work)

'3900380,27643,ARGOS," ",1510,09/02/2005,11/02/2005,02,2005,-56.47,-58.54," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0'

In [8]:
d2 = [d.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for d in data]
d2[:5]

['WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,LON0,SHIP,CRUISE,DATE_,LAT_,LON_,MODEL,FULL_NAME,EMAIL,PROGRAM,COUNTRY,AGE,AGE_WEIGHT,GTS,PROFILES,DMPROFILES,GTSPRESS,GREYLIST,CYCLE_TIME,DRIFT_PRESS,PROFILE_PRESS,OXYGEN,FLUO,TURBIDITY,RAFOS,CDOM,BEAM,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED',
 '3900380,27643,ARGOS," ",1510,09/02/2005,11/02/2005,02,2005,-56.47,-58.54," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900381,27645,ARGOS," ",1511,09/02/2005,11/02/2005,02,2005,-58.78,-58.77," "," ",17/09/2009,-50.7143,33.9768,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,1681.1643,0,1,168,168,2003,0,240,1500,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0',
 '3900382,27652,ARGOS," ",1512,09/02/2005,11/02/2005,02,2005,-57.49,-58.9," "," ",,0,0,APEX,"Turton, Jon",jon.turton@metoffice.gov.uk,Argo UK,UNITED KINGDOM,0,0,0,0,0,0,0,240,

In [9]:
with open('d2.csv', 'w') as f:
    for row in d2:
        f.write(row + '\n')

df = pd.read_csv('d2.csv')
df[:5]

,WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,...,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED
0,3900380,27643,ARGOS,,1510,09/02/2005,11/02/2005,2,2005,-56.4700,...,0,0,0,0,0,0,0,0,0,0
1,3900381,27645,ARGOS,,1511,09/02/2005,11/02/2005,2,2005,-58.7800,...,0,0,0,0,0,0,0,0,0,0
2,3900382,27652,ARGOS,,1512,09/02/2005,11/02/2005,2,2005,-57.4900,...,0,0,0,0,0,0,0,0,0,0
3,3900399,46233,ARGOS,1201,,07/02/2005,14/02/2005,2,2005,-25.2461,...,0,0,0,0,0,0,0,0,0,0
4,3900400,46234,ARGOS,1202,,09/02/2005,14/02/2005,2,2005,-22.7369,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#This prints out certain columns (of my choosing) with all the rows still present
fd_oga = df.loc[: , ['WMO' , 'OXYGEN' , 'GREYLIST' , 'AGE' , 'DATE_' , 'LAT_' , 'LON_'] ] # oga = oxygen, grey list, and age
fd_oga[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,3900380,0,0,0.0000,NaN,0.0000,0.0000
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
2,3900382,0,0,0.0000,NaN,0.0000,0.0000
3,3900399,0,1,2001.7359,01/08/2010,-26.0516,-42.4516
4,3900400,0,1,1941.0619,04/06/2010,-26.5448,-42.5100


In [11]:
#This takes the fd_oga table and only shows the rows that have oxygen data
fd_oxygen = fd_oga.loc[ fd_oga.loc[: , 'OXYGEN'] == 1 , :]
fd_oxygen[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
84,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
196,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
317,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
368,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
436,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [12]:
#This takes the fd_oga table and only shows the rows that aren't grey listed     #gl = grey list
fd_gl = fd_oga.loc[ fd_oga.loc[: , 'GREYLIST'] == 0 , :] #gl = grey list
fd_gl[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,3900380,0,0,0.0000,NaN,0.0000,0.0000
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
2,3900382,0,0,0.0000,NaN,0.0000,0.0000
5,5900339,0,0,2781.0015,23/09/2012,-56.0493,-65.4248
6,5900346,0,0,3130.1958,08/09/2013,-58.1230,-22.3040


In [13]:
#This takes the fd_oga table and only shows the rows that are at least 340 days old
fd_age = fd_oga.loc[ fd_oga.loc[: , 'AGE'] >= 340. , :]
fd_age[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
1,3900381,0,0,1681.1643,17/09/2009,-50.7143,33.9768
3,3900399,0,1,2001.7359,01/08/2010,-26.0516,-42.4516
4,3900400,0,1,1941.0619,04/06/2010,-26.5448,-42.5100
5,5900339,0,0,2781.0015,23/09/2012,-56.0493,-65.4248
6,5900346,0,0,3130.1958,08/09/2013,-58.1230,-22.3040


In [14]:
#Merging the fd_oxygen table with the fd_gl table
fd_merge_og = pd.merge(fd_oxygen , fd_gl) #og = oxygen and grey list
fd_merge_og[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
1,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
2,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
3,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
4,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [15]:
#Later work will go off of this table

#FINALLY HAVE ONLY THE DATA I DESIRE
#PRAISE HELIX
#After all the trial and error (=^.^=)

fd_merge_oga = pd.merge(fd_merge_og , fd_age)
fd_merge_oga[:5]
#Merged the fd_merge_og table with the fd_age table

#Inactive -> active = 0

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,5900245,1,0,1097.1958,05/01/2006,-36.2894,165.6669
1,4900637,1,0,1640.5049,21/08/2009,45.9560,-128.0490
2,4900497,1,0,1030.1457,26/02/2007,34.7538,-63.6900
3,2900460,1,0,792.1306,26/04/2007,44.4140,-153.7139
4,4900651,1,0,1200.1958,12/11/2008,38.4279,-157.9447


In [16]:
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines()
gd[:10]

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150727162538\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [17]:
#May or may not even need this
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines() #gd = global data

gd1 = [g.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for g in gd]

gd1[:10]
#Now to get it so when a user inputs a float number, the DAC of that number shows up

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150727162538\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [18]:
with open('gd1.csv', 'w') as f:
    for row in gd1:
        f.write(row + '\n')

gd_table = pd.read_csv('gd1.csv' , comment = '#')
gd_table[:5]

,file,profiler_type,institution,date_update
0,aoml/13857/13857_meta.nc,845,AO,20120521144513
1,aoml/13858/13858_meta.nc,845,AO,20130222100319
2,aoml/13859/13859_meta.nc,845,AO,20120521144513
3,aoml/15819/15819_meta.nc,845,AO,20120521144513
4,aoml/15820/15820_meta.nc,845,AO,20120521144513


In [19]:
# Would like to read off Internet
gd_table.loc[:,['file']][:10]

,file
0,aoml/13857/13857_meta.nc
1,aoml/13858/13858_meta.nc
2,aoml/13859/13859_meta.nc
3,aoml/15819/15819_meta.nc
4,aoml/15820/15820_meta.nc
5,aoml/15851/15851_meta.nc
6,aoml/15852/15852_meta.nc
7,aoml/15853/15853_meta.nc
8,aoml/15854/15854_meta.nc
9,aoml/15855/15855_meta.nc


In [20]:
bList =[12,35,67,98]
for i in range(50):
    if i in bList:
        print i, "in bList"

12 in bList
35 in bList


In [25]:
good_floats = []
for index,row in fd_merge_oga.loc[:,['WMO']].iterrows():
    good_floats.append(row['WMO'])
print good_floats[:10]

['5900245', '4900637', '4900497', '2900460', '4900651', '4900652', '5900420', '5900421', '5900422', '3900274']


In [24]:
dac_substring = []
for index,row in gd_table.loc[:,['file']].iterrows():
    floatNum = row['file'].split('/')[1]
    all_floats.append(floatNum)

    if floatNum in good_floats:
        dac_substring.append(row['file'])
    

NameError: name 'all_floats' is not defined

In [ ]:
print dac_substring[:10]

In [23]:
tdsUrl = ''
for dac in dac_substring:
    tdsUrl += "http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/"
    tdsUrl += '/'.join(dac.split('/')[:2])
    tdsUrl += "/profiles/catalog.xml"
    print tdsUrl
    break

NameError: name 'dac_substring' is not defined

In [22]:
c = Crawl("http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/coriolis/1900650/profiles/catalog.xml")
urls = [s.get("url") for d in c.datasets for s in d.services if s.get("service").lower() == "opendap"]

In [ ]:
def conv

In [ ]:
floatLookup = {}
for url in urls:
    print url
    ds = Dataset(url)
    floatNum = url.split('/')[9]
    print floatNum
    p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], 
               ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])
    lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])
    
    # Convert o from micromol/kg to ml/l 
    ##perc_sat = 100 * convertTomll(o) / satO2(s,t) 
    print s,t,perc_sat
    floatLookup[floatNum] = (p,t,s,o,lat,lon,mtime)

In [ ]:
floatLookup

In [ ]:
p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])

In [ ]:
p,t,s,o

In [35]:
lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])

In [36]:
lat,lon,mtime

(6.0310000000000006, -14.026, 21985.069212962964)

In [115]:
dens_formula(p , t , s)

1039.220347679988

In [28]:
seawater.eos80.dens0(s, t)

NameError: name 't' is not defined